This script is used to produce Figure 6 and Figure S4 (change in grid-cell specific PPP for 2010 and 2012) for the Payne et al. paper "Quantifying the potential predictability of Arctic primary production". In order to reproduce these figure, the script will need to be run both for 2010 and 2030 data, outputting netcdf files of PPP for each.

1. Import Packages

In [1]:
import xarray as xr
import numpy as np
import numpy.ma as ma
import glob
import math
import cmocean
import shapefile as shp
import matplotlib.pyplot as plt
from  matplotlib.cm import ScalarMappable
from  matplotlib.colors import ListedColormap, BoundaryNorm
from matplotlib import colors as c
import cartopy.crs as ccrs
import cartopy.feature as cfeature

2. Select initialization that we want to calculate PPP for. In Payne et al., May initialization (imno = 2) was chosen.

In [104]:
# a. Alter imno to change the initialization month and yr to change the year of analysis.
imno = 2
yr = '2030' #### Change between 2010 and 2030
imstr = ['01', '03', '05', '07', '09', '11']
im = imstr[imno]

# b. This then sets the start and end month correctly, which you'll need to use to
#    read in the correct files.
for i in np.arange(0,6,1):
    if yr == '2010':
        stmon = '2010' + im
        if im == imstr[0]:
            enmon = '201112'
        elif im == imstr[i]:
            enmon = '2012' + str(i*2).zfill(2)
    else:
        stmon = '2030' + im
        if im == imstr[0]:
            enmon = '203112'
        elif im == imstr[i]:
            enmon = '2032' + str(i*2).zfill(2)

3. Read in NPP files for diatom, diazotroph, and small phytoplankton functional types (photoC_diat_zint, photoC_diaz_zint, and photoC_sp_zint, respectively) from the CESM2-LE for years 2010-2015. NPP is calculated for each model grid cell and across all ensemble members for the years 2010-2012 (dat_climt). Here, we also calculate a climatology and variance for log(NPP) (clim_log and clim_logvar), used to calculate PPP for each grid cell.

In [105]:
# a. Set the variable (vnam) and whether or not it's an NPP variable (T/F).
vnam = ['photoC_diat_zint','photoC_diaz_zint','photoC_sp_zint']#'SST'#'diat_light_lim_Cweight_avg_100m'#['photoC_diat_zint','photoC_diaz_zint','photoC_sp_zint']
vdir = 'ocn'
vfil = ['.pop.h.photoC_diat_zint.','.pop.h.photoC_diaz_zint.','.pop.h.photoC_sp_zint.']
NPP = True#False
if vnam[0] == 'photoC_diat_zint': # NPP has to be summed across phytoplankton functional types
    vn = 3
else:
    vn = 1

# b. Select variable of interest and locate the files
dat_clim = np.full([50, 36, 384, 320], np.nan)
for j in np.arange(0,vn):
    if NPP == True:
        fdir = '/glade/campaign/cgd/cesm/CESM2-LE/'+ vdir + '/proc/tseries/month_1/' + vnam[j] + '/'
    else:
        fdir = '/glade/campaign/cgd/cesm/CESM2-LE/'+ vdir + '/proc/tseries/month_1/' + vnam + '/'

# c. Make an array of nans to store the values from all EMs
    i = -1

# d. Loop through all the files in the directory, storing data in dat_clim
    for file in glob.glob(fdir + "*smbb*202501*"): #### Change between '*201001*' and '*202501*'
        i = i + 1

# e. Open the netcdf and store the variable of interest as dat. Also open lon and lat.
        ncfile = xr.open_dataset(file) 
        if NPP == True:
            dat = ncfile[vnam[j]].values
        else:
            dat = ncfile[vnam].values
        lon, lat = ncfile.TLONG.values, ncfile.TLAT.values #if in 'ice', use 'TLON', in 'ocn', use 'TLONG'

# f. Interpolate lat and lon values over holes in the arrays
        ok = ~np.isnan(lon)
        xp = ok.ravel().nonzero()[0]
        fp = lon[~np.isnan(lon)]
        x  = np.isnan(lon).ravel().nonzero()[0]
        lon[np.isnan(lon)] = np.interp(x, xp, fp)

        ok = ~np.isnan(lat)
        xp = ok.ravel().nonzero()[0]
        fp = lat[~np.isnan(lat)]
        x  = np.isnan(lat).ravel().nonzero()[0]
        lat[np.isnan(lat)] = np.interp(x, xp, fp)

# g. Loop through the ensemble members, assigning each to its own column.
        if j == 0:
            for t in range(0,36): 
                dat_clim[i,t,:,:] = dat[t+60,:,:]
                #### In 2010, this should read dat[t,:,:]. In 2030, it reads dat[t+60,:,:] to skip 2025-2029
        else:
            for t in range(0,36):
                dat_clim[i,t,:,:] = dat[t+60,:,:] + dat_clim[i,t,:,:]
                #### In 2010, this should read dat[t,:,:]. In 2030, it reads dat[t+60,:,:] to skip 2025-2029

# h. Sum over EMs, then divide by 50 to get a climatology for each grid cell
dat_climt = np.full([150, 12, 384, 320], np.nan)
dat_climt_log = np.full([150, 12, 384, 320], np.nan)
clim = np.full([12, 384, 320], np.nan);
clim_sd = np.full([12, 384, 320], np.nan)
clim_var = np.full([12, 384, 320], np.nan)
clim_log = np.full([12, 384, 320], np.nan)
clim_logvar = np.full([12, 384, 320], np.nan)
   
for t in range(0,12):
    dat_climt[0:50,t,:,:] = dat_clim[:,t,:,:]
    print('done')
    dat_climt[50:100,t,:,:] = dat_clim[:,t + 12,:,:]
    dat_climt[100:150,t,:,:] = dat_clim[:,t + 24,:,:]
    dat_climt_log[:,t,:,:] = np.log(dat_climt[:,t,:,:] + 0.000000000000001)
    clim[t,:,:] = (np.nansum(dat_climt[:,t,:,:], axis = 0)) / 150
    clim_sd[t,:,:] = np.nanstd(dat_climt[:,t,:,:], axis = 0)
    clim_var[t,:,:] = np.nanvar(dat_climt[:,t,:,:], axis = 0)
    clim_log[t,:,:] = (np.nansum(dat_climt_log[:,t,:,:], axis = 0)) / 150
    clim_logvar[t,:,:] = np.nanvar(dat_climt_log[:,t,:,:], axis = 0)
    
    for i in np.arange(0,384):
        for j in np.arange(0,320):
            if clim_var[t,i,j] < .000001:
                clim_var[t,i,j] = np.nan
            if clim_logvar[t,i,j] < .000001:
                clim_logvar[t,i,j] = np.nan


done


/glade/u/apps/opt/conda/envs/npl-2023b/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1872: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/glade/derecho/scratch/cpayne/tmp/ipykernel_7159/1772014948.py:74: RuntimeWarning: Degrees of freedom <= 0 for slice.
  clim_var[t,:,:] = np.nanvar(dat_climt[:,t,:,:], axis = 0)
/glade/derecho/scratch/cpayne/tmp/ipykernel_7159/1772014948.py:76: RuntimeWarning: Degrees of freedom <= 0 for slice.
  clim_logvar[t,:,:] = np.nanvar(dat_climt_log[:,t,:,:], axis = 0)


done
done
done
done
done
done
done
done
done
done
done


4. Calculate the area of each grid cell in the Arctic (here, >50°N). This variable is used to scale NPP by area and also to exclude sub-Arctic grid cells.

In [4]:
# a. Read in the areacello file and use this to weight each variable by the area of the grid cell.
acdir = '/glade/collections/cmip/CMIP6/CMIP/NCAR/CESM2/historical/r1i1p1f1/Ofx/areacello/gn/files/d20190308/areacello_Ofx_CESM2_historical_r1i1p1f1_gn.nc'
acfil = xr.open_dataset(acdir)
areacello = acfil['areacello'].values # tarea <- ocean files
areacello_ocn = np.full([384,320], np.nan)
for i in np.arange(0,384):
    for j in np.arange(0,320):
        if np.isnan(dat_climt[0,2,i,j]) == False and lat[i,j] > 50:
            areacello_ocn[i,j] = areacello[i,j]


/glade/u/apps/opt/conda/envs/npl-2023b/lib/python3.10/site-packages/xarray/conventions.py:431: SerializationWarning: variable 'areacello' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(


5. Read in monthly NPP for the two-year perfect model experiments. These experiments were initialized every other month (6x per year) from five different CESM2-LE ensemble members and have 15 ensemble members in each set. Consequently, the dimensions of pmeall are initialization, initial ensemble member, ensemble member, time, longitude, latitude.

In [106]:
# a. Initialize arrays
pmeall = np.full([5,15,24,384,320], np.nan)
pmeall_log = np.full([5,15,24,384,320], np.nan)
pme_var = np.full([5,24,384,320], np.nan)
pme_mean = np.full([5,24,384,320], np.nan)

# b. Set up big loop for opening arrays and calculating variance
i = -1
for init in np.arange(0,5):
    if init == 0:
        l = ''
    else:
        i = i + 1
        let = ['b', 'c', 'd', 'e']
        l = let[i]
# c. Loop through the ensemble members. Open the appropriate file and add it to the array
    for ens in np.arange(0,15):
        i1 = ens + 1
        ### To switch between 2010 and 2030, change the first part of pmedir to either
        #/glade/campaign/cgd/ppc/mholland/predict_cesm2/b.e21.BHIST_CESM2_2010_      or
        #/glade/campaign/cgd/ppc/mholland/predict_cesm2_2030/b.e21.BSSP370smbb_CESM2_2030_
        pmedir = '/glade/campaign/cgd/ppc/mholland/predict_cesm2_2030/b.e21.BSSP370smbb_CESM2_2030_' + im + '_ens00'+ str(i1) + str(l) +'/' + vdir + '/proc/tseries/month_1/'
        if NPP == True: # if NPP is the variable, loop through all phytoplankton groups and sum them
            for varloop in np.arange(0,len(vdir)):
                ### To switch between 2010 and 2030, change the first part of the file name (following pmedir) to
                # 'b.e21.BHIST_CESM2_2010_' or 'b.e21.BSSP370smbb_CESM2_2030_'.
                pmef = pmedir + 'b.e21.BSSP370smbb_CESM2_2030_' + im + '_ens00' + str(i1) + str(l) + vfil[varloop] + stmon + '-' + enmon + '.nc'
                pme = xr.open_dataset(pmef)
                if varloop == 0:
                    pmeall[init,ens,:,:,:] = pme[vnam[varloop]][:,:,:]
                else:
                    pmeall[init,ens,:,:,:] = pmeall[init,ens,:,:,:] + pme[vnam[varloop]][:,:,:]
            vname = 'photoC_sp_zint'
        else:
            ### To switch between 2010 and 2030, change the first part of the file name (following pmedir) to
            # 'b.e21.BHIST_CESM2_2010_' or 'b.e21.BSSP370smbb_CESM2_2030_'.
            pmef = pmedir + 'b.e21.BSSP370smbb_CESM2_2030_' + im + '_ens00' + str(i1) + str(l) + vfil + stmon + '-' + enmon + '.nc'
            pme = xr.open_dataset(pmef)
            pmeall[init,ens,:,:,:] = pme[vnam][:,:,:]
            vname = vnam
# d. Calculate variability across the perfect model ensemble members
pmeall_log[:,:,:,:,:] = np.log(pmeall[:,:,:,:,:] + 0.000000000000001)
pme_var[:,:,:,:] = np.nanvar(pmeall_log[:,:,:,:,:], axis = 1)
pme_mean[:,:,:,:] = np.nanmean(pmeall_log[:,:,:,:,:], axis = 1)


/glade/derecho/scratch/cpayne/tmp/ipykernel_7159/1282205685.py:43: RuntimeWarning: Degrees of freedom <= 0 for slice.
  pme_var[:,:,:,:] = np.nanvar(pmeall_log[:,:,:,:,:], axis = 1)
/glade/derecho/scratch/cpayne/tmp/ipykernel_7159/1282205685.py:44: RuntimeWarning: Mean of empty slice
  pme_mean[:,:,:,:] = np.nanmean(pmeall_log[:,:,:,:,:], axis = 1)


6. Calculate PPP across the Arctic for the two years of each initialization. PPP is calculated using log(NPP), and these equations are provided by Campbell (1995). Variance for the perfect model experiments is calculated as a mean across the five different ensemble members the forecast simulations were initialized from.

In [107]:
# a. Mask out values outside of the "Arctic Circle", defined here as < 50°N, and initialize arrays
arcirc = ma.masked_array(pme[vname][0,:,:], pme["TLAT"][:,:] < 50)
PPP_mean = np.full([24,384,320],np.nan)
var_clim = np.full([12,384,320],np.nan); var_pme = np.full([24,384,320],np.nan)
for i in np.arange(0,384):
    if i > 300 and i%10 == 0:
        print(i)
    for j in np.arange(0,320):

# b. If we are in the "Arctic Circle" and there are values for the perfect model experiments, calculate PPP.
        if (arcirc[i,j] >= 0 & np.isnan(pme_var[0,5,i,j]) == False):
            for t in np.arange(0,12):

# c. Calculate the climatology "month" (tclim)
                if (t + imno*2) < 12:
                    tclim = t + imno * 2
                else:
                    tclim = t + imno * 2 - 12
                    
# d. Calculate the mean and variance for the climatology
                meanlog_clim = clim_log[tclim,i,j]
                varlog_clim = clim_logvar[tclim,i,j]
                var_clim[t,i,j] = (math.exp(varlog_clim) - 1) * (math.exp(2* meanlog_clim + varlog_clim))

# e. Calculate the mean and variance for the perfect model experiments for t, then calculate PPP.
                meanlog_pme = np.nanmean(pme_mean[:,t,i,j])
                varlog_pme = np.nanmean(pme_var[:,t,i,j])
                var_pme[t,i,j] = (math.exp(varlog_pme) - 1) * (math.exp(2* meanlog_pme + varlog_pme))
                PPP_mean[t,i,j] = 1 - (varlog_pme / varlog_clim)
                
# f. Calculate the mean and variance for the perfect model experiments for t + 12, then calculate PPP.
                meanlog_pme = np.nanmean(pme_mean[:,t + 12,i,j])
                varlog_pme = np.nanmean(pme_var[:,t + 12,i,j])
                var_pme[t+12,i,j] = (math.exp(varlog_pme) - 1) * (math.exp(2* meanlog_pme + varlog_pme))
                PPP_mean[t+12,i,j] = 1 - (varlog_pme / varlog_clim)


310
320
330
340
350
360
370
380


In [108]:
var_pme_2030 = var_pme
var_clim_2030 = var_clim

7. Write out PPP values as netcdf. Then read these back in for the desired years and initialization months. Repeat steps 1-7a as often as desired, then continue on to 7b to plot figures.

In [ ]:
# a. Write out PPP values as a netcdf.
cdir = "/glade/u/home/cpayne/PredictPerfect/Output/PPPmean_" + vnnam + im + yr + ".nc"
ds = xr.Dataset(
    data_vars=dict(
        PPP=(["time", "lat", "lon"], PPP_mean),
    ),
    coords=dict(
        time=(pme.time.values),
        TLAT=(["lat", "lon"], lat),
        TLON=(["lat", "lon"], lon),
    ),
)
ds.to_netcdf(path = cdir, mode = 'w')

In [17]:
# b. Read in netcdf files of PPP for different years and initialization months.
PPP012010 = xr.open_dataset("/glade/u/home/cpayne/Projects/PredictPerfect/Output/PPPmean_NPP012010.nc")
PPP012030 = xr.open_dataset("/glade/u/home/cpayne/Projects/PredictPerfect/Output/PPPmean_NPP012030.nc")
PPP032010 = xr.open_dataset("/glade/u/home/cpayne/Projects/PredictPerfect/Output/PPPmean_NPP032010.nc")
PPP032030 = xr.open_dataset("/glade/u/home/cpayne/Projects/PredictPerfect/Output/PPPmean_NPP032030.nc")
PPP052010 = xr.open_dataset("/glade/u/home/cpayne/Projects/PredictPerfect/Output/PPPmean_NPP052010.nc")
PPP052030 = xr.open_dataset("/glade/u/home/cpayne/Projects/PredictPerfect/Output/PPPmean_NPP052030.nc")
PPP072010 = xr.open_dataset("/glade/u/home/cpayne/Projects/PredictPerfect/Output/PPPmean_NPP072010.nc")
PPP072030 = xr.open_dataset("/glade/u/home/cpayne/Projects/PredictPerfect/Output/PPPmean_NPP072030.nc")
PPP092010 = xr.open_dataset("/glade/u/home/cpayne/Projects/PredictPerfect/Output/PPPmean_NPP092010.nc")
PPP092030 = xr.open_dataset("/glade/u/home/cpayne/Projects/PredictPerfect/Output/PPPmean_NPP092030.nc")
PPP112010 = xr.open_dataset("/glade/u/home/cpayne/Projects/PredictPerfect/Output/PPPmean_NPP112010.nc")
PPP112030 = xr.open_dataset("/glade/u/home/cpayne/Projects/PredictPerfect/Output/PPPmean_NPP112030.nc")


In [18]:
for i in np.arange(0,384):
    if i > 299 and i%20 == 0:
        print(i)
    for j in np.arange(0,320):
        if np.isnan(areacello_ocn[i,j]) == False:
            for t in np.arange(0,24):
                if PPP012010['PPP'][t,i,j] < 0:
                    PPP012010['PPP'][t,i,j] = np.nan
                if PPP012030['PPP'][t,i,j] < 0:
                    PPP012030['PPP'][t,i,j] = np.nan
                if PPP032010['PPP'][t,i,j] < 0:
                    PPP032010['PPP'][t,i,j] = np.nan
                if PPP032030['PPP'][t,i,j] < 0:
                    PPP032030['PPP'][t,i,j] = np.nan
                if PPP052010['PPP'][t,i,j] < 0:
                    PPP052010['PPP'][t,i,j] = np.nan
                if PPP052030['PPP'][t,i,j] < 0:
                    PPP052030['PPP'][t,i,j] = np.nan
                if PPP072010['PPP'][t,i,j] < 0:
                    PPP072010['PPP'][t,i,j] = np.nan
                if PPP072030['PPP'][t,i,j] < 0:
                    PPP072030['PPP'][t,i,j] = np.nan
                if PPP092010['PPP'][t,i,j] < 0:
                    PPP092010['PPP'][t,i,j] = np.nan
                if PPP092030['PPP'][t,i,j] < 0:
                    PPP092030['PPP'][t,i,j] = np.nan
                if PPP112010['PPP'][t,i,j] < 0:
                    PPP112010['PPP'][t,i,j] = np.nan
                if PPP112030['PPP'][t,i,j] < 0:
                    PPP112030['PPP'][t,i,j] = np.nan

300
320
340
360
380


8. Plot change in PPP between the two netcdf PPP files. Change the chosen t value to look how this changes over different months.

In [14]:
# a. Read in a shapefile of the LME lines (sf) and LME number for each grid cell.
sf = shp.Reader("/glade/work/kristenk/fish-offline/fish-offline/notebooks/data/LMEs66_Offshore")


In [ ]:
# b. Change t to change what time we are looking at. Plot PPP changes between 2010 and 2030
t = 13
plt.figure(figsize=(4,3)) #plt.figure(figsize=(2.5,2.5))
ax = plt.axes(projection=ccrs.NorthPolarStereo()); 
ax.set_extent([-180, 180, 49, 90], crs = ccrs.PlateCarree());
clim_ac = ma.masked_array(PPP052010["PPP"][t,:,:], PPP052010["TLAT"] < 50)
pc = ax.pcolormesh(ncfile["TLONG"], ncfile["TLAT"], clim_ac,
                    transform=ccrs.PlateCarree(), vmin = -1, vmax = 1, cmap = 'cmo.balance');
ax.coastlines();  ax.add_feature(cfeature.LAND); plt.colorbar(pc);

# c. Plot the lines of the LMEs.
for shape in sf.shapeRecords():
    x = [i[0] for i in shape.shape.points[:]]
    y = [i[1] for i in shape.shape.points[:]]
    ax.plot(x,y,transform=ccrs.PlateCarree(),color='black',linewidth=1)

# d. Write out that figure
# figdir = '/glade/u/home/cpayne/Projects/PredictPerfect/Figures/PaperFigs/deltaPPPmean' + str(mt + 16) + '.png';
plt.savefig(figdir,facecolor='none', dpi = 600);
# plt.close()